In [1]:
import sys
from pathlib import Path
# Get the repository root (2 levels up from current notebook)
repo_root = Path(__file__).parent.parent.parent if '__file__' in globals() else Path.cwd().parent.parent
sys.path.insert(0, str(repo_root))

# import necessary libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, ConfusionMatrixDisplay, precision_score, recall_score, f1_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
import Template
# Definición de modulos

import torch
import numpy             as np
import pandas            as pd
import seaborn           as sns
import matplotlib.pyplot as plt
from xgboost import XGBClassifier

from imblearn.over_sampling  import SMOTE, ADASYN
from sklearn.ensemble        import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics         import confusion_matrix, ConfusionMatrixDisplay, balanced_accuracy_score, recall_score, f1_score, precision_score
from sklearn.decomposition import PCA


Template = Template.Template()

# Build the path to the Excel file in the repository root
data_path = os.path.join(os.path.dirname(os.getcwd()), '..', 'Datos1_InteraccionesNIR.xlsx')

In [2]:
# Build the path to the Excel file in the repository root
data_path = os.path.join(repo_root, 'Datos1_InteraccionesNIR.xlsx')
# import the data from all sheets of the Excel file

df12 = pd.read_excel(data_path, sheet_name=12)


## Classification of Healthy plants vs Unhealthy plants

## SMOTE

In [3]:
# Make a new column 'Health_Status'
df = df12.copy()

# Drop the 'Planta' column
df = df.drop(columns=['Planta'])

columns_idx = list(df.columns)[1:]

# Add 'Sana' column at the beginning (after 'Tratamiento' and 'Planta')
df.insert(1, 'Sana', df['Tratamiento'].apply(lambda x: 1 if x == 'Control' else 0))

df.head()

,Tratamiento,Sana,350,351,352,353,354,355,356,357,...,2491,2492,2493,2494,2495,2496,2497,2498,2499,2500
0,Control,1,0.068232,0.072210,0.066953,0.071272,0.069221,0.061493,0.060840,0.058393,...,0.038243,0.038003,0.037529,0.037242,0.036899,0.036715,0.036668,0.036646,0.036726,0.036748
1,Control,1,0.069469,0.068557,0.067614,0.072405,0.067971,0.057739,0.057577,0.056457,...,0.034358,0.034108,0.033718,0.033456,0.033050,0.032910,0.033039,0.033020,0.033061,0.032994
2,Control,1,0.068977,0.068584,0.061503,0.067413,0.064611,0.055044,0.057237,0.055094,...,0.034990,0.034736,0.034400,0.034171,0.033795,0.033650,0.033773,0.033704,0.033577,0.033530
3,Control,1,0.068365,0.068669,0.067072,0.064658,0.062009,0.061129,0.064017,0.058342,...,0.032503,0.032323,0.032073,0.031877,0.031593,0.031465,0.031439,0.031375,0.031405,0.031339
4,Control,1,0.072596,0.074704,0.075351,0.071436,0.066297,0.062701,0.062234,0.058701,...,0.036653,0.036470,0.036223,0.036069,0.035908,0.035826,0.035823,0.035858,0.035985,0.035996


In [4]:
# Apply SMOTE to balance the dataset
X = df[columns_idx]
y = df['Sana']

smote = SMOTE(random_state=42)
X_balanced, y_balanced = smote.fit_resample(X, y)

# Create balanced dataframe
df_balanced = pd.DataFrame(X_balanced, columns=columns_idx)
df_balanced.insert(0, 'Sana', y_balanced)

print(f"Original class distribution:\n{y.value_counts()}")
print(f"\nBalanced class distribution:\n{y_balanced.value_counts()}")
print(f"\nOriginal shape: {X.shape}, Balanced shape: {X_balanced.shape}")

df_balanced.head()

Original class distribution:
Sana
0    187
1     30
Name: count, dtype: int64

Balanced class distribution:
Sana
1    187
0    187
Name: count, dtype: int64

Original shape: (217, 2151), Balanced shape: (374, 2151)


C:\Users\ricar\AppData\Local\Temp\ipykernel_28580\2984401751.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_balanced.insert(0, 'Sana', y_balanced)


,Sana,350,351,352,353,354,355,356,357,358,...,2491,2492,2493,2494,2495,2496,2497,2498,2499,2500
0,1,0.068232,0.072210,0.066953,0.071272,0.069221,0.061493,0.060840,0.058393,0.054574,...,0.038243,0.038003,0.037529,0.037242,0.036899,0.036715,0.036668,0.036646,0.036726,0.036748
1,1,0.069469,0.068557,0.067614,0.072405,0.067971,0.057739,0.057577,0.056457,0.055687,...,0.034358,0.034108,0.033718,0.033456,0.033050,0.032910,0.033039,0.033020,0.033061,0.032994
2,1,0.068977,0.068584,0.061503,0.067413,0.064611,0.055044,0.057237,0.055094,0.053115,...,0.034990,0.034736,0.034400,0.034171,0.033795,0.033650,0.033773,0.033704,0.033577,0.033530
3,1,0.068365,0.068669,0.067072,0.064658,0.062009,0.061129,0.064017,0.058342,0.052955,...,0.032503,0.032323,0.032073,0.031877,0.031593,0.031465,0.031439,0.031375,0.031405,0.031339
4,1,0.072596,0.074704,0.075351,0.071436,0.066297,0.062701,0.062234,0.058701,0.057463,...,0.036653,0.036470,0.036223,0.036069,0.035908,0.035826,0.035823,0.035858,0.035985,0.035996


In [5]:
# Add a column to distinguish between data before and after balancing
# Mark original data
n_original = len(df)
n_synthetic = len(df_balanced) - n_original

# Create status array: first n_original are 'Original', rest are 'Balanced'
data_status = ['Original'] * n_original + ['Balanced'] * n_synthetic
df_balanced.insert(0, 'Data_Status', data_status)
df_balanced.head()

C:\Users\ricar\AppData\Local\Temp\ipykernel_28580\56007382.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_balanced.insert(0, 'Data_Status', data_status)


,Data_Status,Sana,350,351,352,353,354,355,356,357,...,2491,2492,2493,2494,2495,2496,2497,2498,2499,2500
0,Original,1,0.068232,0.072210,0.066953,0.071272,0.069221,0.061493,0.060840,0.058393,...,0.038243,0.038003,0.037529,0.037242,0.036899,0.036715,0.036668,0.036646,0.036726,0.036748
1,Original,1,0.069469,0.068557,0.067614,0.072405,0.067971,0.057739,0.057577,0.056457,...,0.034358,0.034108,0.033718,0.033456,0.033050,0.032910,0.033039,0.033020,0.033061,0.032994
2,Original,1,0.068977,0.068584,0.061503,0.067413,0.064611,0.055044,0.057237,0.055094,...,0.034990,0.034736,0.034400,0.034171,0.033795,0.033650,0.033773,0.033704,0.033577,0.033530
3,Original,1,0.068365,0.068669,0.067072,0.064658,0.062009,0.061129,0.064017,0.058342,...,0.032503,0.032323,0.032073,0.031877,0.031593,0.031465,0.031439,0.031375,0.031405,0.031339
4,Original,1,0.072596,0.074704,0.075351,0.071436,0.066297,0.062701,0.062234,0.058701,...,0.036653,0.036470,0.036223,0.036069,0.035908,0.035826,0.035823,0.035858,0.035985,0.035996


In [6]:
# data treatment

# Data preparation for model production
df_X = df_balanced.drop(columns=['Sana', 'Data_Status'])

X = df_X.values  
y = df_balanced['Sana'].values
data_status = df_balanced['Data_Status'].values  # Keep track of data origin

#split the data into training and testing sets
X_train, X_test, y_train, y_test, status_train, status_test = train_test_split(
    X, y, data_status, test_size=0.3, random_state=42, stratify=y
)

# Print distribution
print(f"Test set distribution:")
print(f"Original samples: {sum(status_test == 'Original')}")
print(f"Balanced samples: {sum(status_test == 'Balanced')}")

Test set distribution:
Original samples: 63
Balanced samples: 50


## Logistic Regression

In [7]:
# Define a more comprehensive parameter grid for hyperparameter tuning
param_grid = [
    # L1 regularization options
    {
        'PCA__n_components': [2],
        'model__penalty': ['l1'],
        'model__C': [0.001, 0.01, 0.1, 1, 10, 100],
        'model__solver': ['liblinear', 'saga'],
        'model__class_weight': [None, 'balanced']
    },
    # L2 regularization options
    {
        'PCA__n_components': [2],
        'model__penalty': ['l2'],
        'model__C': [0.001, 0.01, 0.1, 1, 10, 100],
        'model__solver': ['liblinear', 'saga', 'lbfgs'],
        'model__class_weight': [None, 'balanced']
    },
    # Elasticnet option (saga only)
    {
        'PCA__n_components': [2],
        'model__penalty': ['elasticnet'],
        'model__C': [0.001, 0.01, 0.1, 1, 10, 100],
        'model__solver': ['saga'],
        'model__l1_ratio': [0.2, 0.5, 0.8],
        'model__class_weight': [None, 'balanced']
    }
]

# Create a pipeline
pipeline = Pipeline([
    ('PCA', PCA()),
    ('model', LogisticRegression(max_iter=1000))
])

grid_search = GridSearchCV(pipeline, param_grid, cv=5, n_jobs=-1, scoring='accuracy')
grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_
# Make predictions with the best model
y_pred_best = best_model.predict(X_test)
# Evaluate the best model
print("Best Model Accuracy:", accuracy_score(y_test, y_pred_best))
print("Best Model Classification Report:\n", classification_report(y_test, y_pred_best))

Best Model Accuracy: 0.8761061946902655
Best Model Classification Report:
               precision    recall  f1-score   support

           0       0.91      0.84      0.87        57
           1       0.85      0.91      0.88        56

    accuracy                           0.88       113
   macro avg       0.88      0.88      0.88       113
weighted avg       0.88      0.88      0.88       113



In [8]:
# Parameter for the best model found
best_params = grid_search.best_params_
print("Best Parameters:", best_params)

Best Parameters: {'PCA__n_components': 2, 'model__C': 1, 'model__class_weight': None, 'model__penalty': 'l1', 'model__solver': 'liblinear'}


In [9]:
cm = confusion_matrix(y_test, y_pred_best)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=best_model.classes_)
# Display the confusion matrix
disp.plot(cmap='Blues')
plt.title(f'Confusion Matrix - Logistic Regression with PCA - accuracy: {accuracy_score(y_test, y_pred_best):.4f}')
plt.tight_layout()
plt.show()

In [10]:
# Detailed analysis of predictions by data origin
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

# Separate predictions by data origin
original_mask = status_test == 'Original'
balanced_mask = status_test == 'Balanced'

# Calculate metrics for original data
y_test_original = y_test[original_mask]
y_pred_original = y_pred_best[original_mask]

# Calculate metrics for balanced data
y_test_balanced = y_test[balanced_mask]
y_pred_balanced = y_pred_best[balanced_mask]

# Create confusion matrices
cm_original = confusion_matrix(y_test_original, y_pred_original)
cm_balanced = confusion_matrix(y_test_balanced, y_pred_balanced)
cm_total = confusion_matrix(y_test, y_pred_best)

# Calculate accuracies
acc_original = accuracy_score(y_test_original, y_pred_original)
acc_balanced = accuracy_score(y_test_balanced, y_pred_balanced)
acc_total = accuracy_score(y_test, y_pred_best)

# Create figure with subplots
fig = plt.figure(figsize=(18, 5))
gs = GridSpec(1, 3, figure=fig)

# Plot 1: Original data confusion matrix
ax1 = fig.add_subplot(gs[0, 0])
disp1 = ConfusionMatrixDisplay(confusion_matrix=cm_original, display_labels=['Unhealthy', 'Healthy'])
disp1.plot(cmap='Greens', ax=ax1, colorbar=False)
ax1.set_title(f'Original Data Only\nAccuracy: {acc_original:.4f}\n(n={sum(original_mask)})', fontsize=12)

# Plot 2: Balanced (synthetic) data confusion matrix
ax2 = fig.add_subplot(gs[0, 1])
disp2 = ConfusionMatrixDisplay(confusion_matrix=cm_balanced, display_labels=['Unhealthy', 'Healthy'])
disp2.plot(cmap='Oranges', ax=ax2, colorbar=False)
ax2.set_title(f'Synthetic Data Only\nAccuracy: {acc_balanced:.4f}\n(n={sum(balanced_mask)})', fontsize=12)

# Plot 3: Total confusion matrix
ax3 = fig.add_subplot(gs[0, 2])
disp3 = ConfusionMatrixDisplay(confusion_matrix=cm_total, display_labels=['Unhealthy', 'Healthy'])
disp3.plot(cmap='Blues', ax=ax3, colorbar=False)
ax3.set_title(f'All Test Data\nAccuracy: {acc_total:.4f}\n(n={len(y_test)})', fontsize=12)

plt.tight_layout()
plt.show()